In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import data_tool as dt
import time

epoch_count = 30

In [3]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [4]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [5]:
ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [6]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available:  []
Is the Tensor on GPU #0:   False


In [7]:
def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 243.91ms


In [8]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [5]:
L = 50
T = 100
phi = dt.get_random_binary_signature_matrix(T, L)

In [7]:
Y, _, _ = dt.get_dataset(1000, phi)

In [ ]:
l = 50
total = 100
batch_size = 1000
p = 0.1
phi = np.array(get_matrix(total, l), 'float32')
ave_power = get_avege_power(p,phi,total,100000)
# Training Parameters
learning_rate = 0.001
display_step = 1000
train_sigma = get_sigma_by_snr(10, ave_power, l)
print(ave_power)
# Network Parameters
num_input = l # input of decoder
num_hidden_decoder = (l + total)# decoder layer num features
num_output = total # output of decoder or this NN

In [ ]:
# gpu_options = tf.GPUOptions(allow_growth=True)
def get_sigma_by_snr(snr, ave_power, signal_l):
    sigma2 = ave_power / (signal_l * np.power(10.0, snr/10.0))
    sigma = np.sqrt(sigma2)
    return sigma

def get_final_result_for_detector(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.5:
            r[i] = 0
    return r

def get_final_result_for_decoder(output):
    r = np.ones_like(output[0])
    for i in range(output[0].size):
        if output[0][i] <= 0.05:
            r[i] = 0
    return r

def get_matrix(size, length):
    ma = 1-2*np.random.binomial(1,0.5, (size,length))
    mb = np.random.binomial(1,0.5, (size,length))
    return ma*mb

def get_batch(data_size, p, phi, total):
    phi = np.transpose(phi)
    x = np.random.binomial(n=1, p=p, size=(total,data_size))
    h_x = np.random.rayleigh(1, size=(total,data_size)) * x
    phi_h_x = np.dot(phi, h_x)
    phi_x = np.dot(phi, x)
    return np.transpose(phi_x), np.transpose(x), np.transpose(phi_h_x), np.transpose(h_x)

def get_avege_power(p, phi, total, test_time):
    phi = np.transpose(phi)
    sum=0
    for i in range(test_time):
        x = np.random.binomial(n=1, p=p, size=(total,1))
        phi_x = np.dot(phi, x)
        sum += np.power(np.linalg.norm(phi_x, ord=2), 2)
    return sum/test_time